In [23]:
import dash
import dash_core_components as dcc 
import dash_html_components as html 
import dash_table  
from dash.dependencies import Input, Output
import plotly.graph_objs as go 
import plotly.figure_factory as ffimport 
import pandas as pd
import plotly.graph_objs as go
import numpy as np
import itertools
import ccxt
import json
import datetime as dt 

# Define my world
spot_exchanges = ['bitfinex','bitstamp','coinbasepro','kraken','liquid','gemini','binance',
                    'bitbank','huobipro','poloniex','bithumb','bittrex','kucoin2']

exch_dict={}
for x in spot_exchanges:
    exec('exch_dict[x]=ccxt.{}()'.format(x))

Xpto= ['BTC','ETH','XRP','XMR','BCH','EOS','USDT','USDC','TRX','XLM','BSV','XBT','CSP','DAI']
Fiat=['USD','EUR','GBP','CHF','HKD','JPY','CNH','KRW']
xpto_fiat = [xpto+'/'+ fiat for xpto in Xpto for fiat in Fiat]
xpto_xpto = [p[0]+'/'+p[1] for p in itertools.permutations(Xpto,2)]

all_pairs = set(sum(itertools.chain([*exch_dict[x].load_markets()] for x in exch_dict),[])) 
pairs = list(set(xpto_fiat + xpto_xpto) & set(all_pairs))
pairs.sort()

def get_exchanges_for_pair(pair):
    '''input: a pair
    output: a dictionary of ccxt exchange objects of the exchanges listing the pair
    '''
    return {x:exch_dict[x] for x in exch_dict if pair in list(exch_dict[x].load_markets().keys())}
def get_pairs_for_exchange(ex):
    '''input: an exchange
    output: a list of pairs '''
    d={}
    exec('d[ex]=ccxt.{}()'.format(ex))
    d[ex].load_markets()
    return d[ex].symbols

def get_order_books(pair,ex):
    '''pair is the pair string ,'BTC/USD'...
        returns a dictionary of order books for the pair
        special case for binance which API fails if # of parmaeters > 2
    '''
    nobinance= {key:value for key, value in ex.items() if key != 'binance'and  key != 'bitfinex'}
    order_books = {key: value.fetch_order_book(pair,limit=2000 if key!='bithumb' else 50,
                        params={'full':1,'level':3,'limit_bids':0,'limit_asks':0,'type':'both'})
                        for key,value in nobinance.items() }
    if 'binance' in ex:
        order_books['binance'] =  ex['binance'].fetch_order_book(pair,limit=1000)
    if 'bitfinex' in ex:
        order_books['bitfinex'] =  ex['bitfinex'].fetch_order_book(pair,limit=2000)
    return order_books

def aggregate_order_books(dict_of_order_books):
    '''dict_of_order_books is a dict of ccxt like order_books
        retuns a ccxt like dictionary order book sorted by prices 
    '''
    agg_dict_order_book = {}
    bids = []
    for x in dict_of_order_books:
        for bid in dict_of_order_books[x]['bids']:
            bids.append(bid+[x])
    asks = []
    for x in dict_of_order_books:
        for ask in dict_of_order_books[x]['asks']:
            asks.append(ask+[x])
    agg_dict_order_book['bids'] = (pd.DataFrame(bids)).sort_values(by=0,ascending=False).values.tolist()
    agg_dict_order_book['asks'] = (pd.DataFrame(asks)).sort_values(by=0,ascending=True).values.tolist()
    return agg_dict_order_book

def normalize_order_book(order_book,cutoff=.1,step=.001):
    '''order_book is a dictionary with keys bids asks timestamp datetime ...
    where bids is a list of list [[bid,bid_size]] and 
    asks is a list of list [[ask,ask_size]]
    this is returned by ccxt.'exchange'.fetch_order_book()
    returns a dataframe with columns [ask, ask_size, ask_size_$, cum_ask_size_$, bid_, bid_size, bid_size_$, cum_bid_size_$]
    and an index of shape np.linspace(1 - cutoff,1 + cutoff ,step =.001 ~ 10 bps)  
    '''
    try:
        rounding = int(np.ceil(-np.log(step)/np.log(10)))
        agg = True
    except:
        agg = False
    bid_side = pd.DataFrame(order_book['bids'],columns=['bid','bid_size','exc'])
    bid_side['cum_bid_size'] = bid_side['bid_size'].cumsum()
    ask_side = pd.DataFrame(order_book['asks'],columns=['ask','ask_size','exc'])
    ask_side['cum_ask_size'] = ask_side['ask_size'].cumsum()
    ref = (bid_side['bid'][0]+ask_side['ask'][0])/2
    bid_side['bid%'] = round(bid_side['bid']/ref,rounding) if agg else bid_side['bid']/ref
    ask_side['ask%'] = round(ask_side['ask']/ref,rounding) if agg else ask_side['ask']/ref
    bid_side = bid_side[bid_side['bid%']>=1-cutoff]
    ask_side = ask_side[ask_side['ask%']<=1+cutoff]
    bid_side['bid_size_$'] = bid_side['bid_size']*bid_side['bid']
    bid_side['cum_bid_size_$'] = bid_side['bid_size_$'].cumsum()
    ask_side['ask_size_$'] = ask_side['ask_size']*ask_side['ask']
    ask_side['cum_ask_size_$'] = ask_side['ask_size_$'].cumsum()
    normalized_bids = pd.DataFrame(bid_side.groupby('bid%',sort=False).mean()['bid'])
    normalized_bids.columns = ['bid']
    normalized_bids['bid_size'] = bid_side.groupby('bid%',sort=False).sum()['bid_size']
    normalized_bids['cum_bid_size'] = normalized_bids['bid_size'].cumsum()
    normalized_bids['bid_size_$'] = bid_side.groupby('bid%',sort=False).sum()['bid_size_$']
    normalized_bids['cum_bid_size_$'] = normalized_bids['bid_size_$'].cumsum()
    normalized_bids['average_bid_fill'] = normalized_bids['cum_bid_size_$']/normalized_bids['cum_bid_size']
    normalized_bids['bids_exc']=bid_side.groupby('bid%',sort=False).apply(lambda x: x['exc'].loc[x['bid_size'].idxmax()])
    normalized_asks = pd.DataFrame(ask_side.groupby('ask%',sort=False).mean()['ask'])
    normalized_asks.columns = ['ask']
    normalized_asks['ask_size'] = ask_side.groupby('ask%',sort=False).sum()['ask_size']
    normalized_asks['cum_ask_size'] = normalized_asks['ask_size'].cumsum()
    normalized_asks['ask_size_$'] = ask_side.groupby('ask%',sort=False).sum()['ask_size_$']
    normalized_asks['cum_ask_size_$'] = normalized_asks['ask_size_$'].cumsum()
    normalized_asks['average_ask_fill']=normalized_asks['cum_ask_size_$']/normalized_asks['cum_ask_size']
    normalized_asks['asks_exc']=ask_side.groupby('ask%',sort=False).apply(lambda x: x['exc'].loc[x['ask_size'].idxmax()])
    book=pd.concat([normalized_asks,normalized_bids],sort=False)
    return book

def build_book(order_books,pair,exchanges,cutoff=.1,step=0.001):
    ''' gets order books aggreagtes them then normalizes
        returns a dataframe
    '''
    return normalize_order_book(aggregate_order_books({key:order_books[key] for key in exchanges}),cutoff,step)

def plot_book(order_books,pair, exc, relative=True, currency=True, cutoff=.1):
    ''' plots the order book as a v shape chart '''
    order_book = build_book(order_books,pair,exc,cutoff)
    best_bid = round(order_book['bid'].max(),4)
    best_ask = round(order_book['ask'].min(),4)
    if currency:
        col_to_chart = '_$'
    else:
        col_to_chart = ''
    if relative:
        trace_asks=go.Scatter(x=order_book.index,y=order_book['cum_ask_size'+col_to_chart],
                        name='asks',marker=dict(color='rgba(255,0,0,0.6)'),fill='tozeroy',fillcolor='rgba(255,0,0,0.2)')
        trace_bids=go.Scatter(x=order_book.index,y=order_book['cum_bid_size'+col_to_chart],
                        name='asks',marker=dict(color='rgba(0,0,255,0.6)'),fill='tozeroy',fillcolor='rgba(0,0,255,0.2)')     
    else:
        trace_asks=go.Scatter(x=order_book['ask'].fillna(0)+order_book['bid'].fillna(0),y=order_book['cum_ask_size'+col_to_chart],
                        name='asks',marker=dict(color='rgba(255,0,0,0.6)'),fill='tozeroy',fillcolor='rgba(255,0,0,0.15)')
        trace_bids=go.Scatter(x=order_book['ask'].fillna(0)+order_book['bid'].fillna(0),y=order_book['cum_bid_size'+col_to_chart],
                        name='asks',marker=dict(color='rgba(0,0,255,0.6)'),fill='tozeroy',fillcolor='rgba(0,0,255,0.15)')
        
    layout = go.Layout(title = ' - '.join(exc), xaxis = dict(title= pair +'  ' + str(best_bid)+' - '+ str(best_ask)))
    data=[trace_asks,trace_bids]
    figure = go.Figure(data=data,layout=layout)
    return figure

def plot_depth(order_books,pair, exc, relative=True, currency=True, cutoff=.1):
    if currency:
        col_to_chart = '_$'
    else:
        col_to_chart = ''
    order_book = build_book(order_books,pair,exc,cutoff)
    mid = (order_book['bid'].max()+order_book['ask'].min())/2 if relative else 1
    trace_asks = go.Scatter(x=order_book['cum_ask_size'+col_to_chart],y=order_book['average_ask_fill']/mid,
                        name='ask depth',marker=dict(color='rgba(255,0,0,0.6)'),fill='tozerox',fillcolor='rgba(255,0,0,0.15)')
    trace_bids = go.Scatter(x=-order_book['cum_bid_size'+col_to_chart],y=order_book['average_bid_fill']/mid,
                        name='bid depth',marker=dict(color='rgba(0,0,255,0.6)'),fill='tozerox',fillcolor='rgba(0,0,255,0.15)')
    data = [trace_asks,trace_bids]
    figure = go.Figure(data=data, layout={'title': 'Market Depth'})
    return figure

def order_fill(order_book_df, order_sizes,in_ccy=True):
    '''takes in an order book dataframe and an np.array of order sizes
        with size in currecncy by default else in coin
        returns an np.array of the purchase costs or the sale proceeds of an order
    '''
    average_fills = np.zeros(order_sizes.shape)
    mid=(order_book_df['ask'].min()+order_book_df['bid'].max())/2
    if in_ccy:
        order_sizes=order_sizes/mid
    for i , order_size in enumerate(order_sizes):
        if order_size > 0:
            try:
                last_line = order_book_df[order_book_df['cum_ask_size']>order_size].iloc[0]
                ccy_fill = last_line['cum_ask_size_$']+(order_size-last_line['cum_ask_size'])*last_line['ask']
                average_fill=ccy_fill/order_size
            except:
                average_fill=np.nan
        elif order_size < 0:
            try:
                last_line = order_book_df[order_book_df['cum_bid_size'] > -order_size].iloc[0]
                ccy_fill=last_line['cum_bid_size_$']+(-order_size-last_line['cum_bid_size'])*last_line['bid']
                average_fill = -ccy_fill/order_size
            except:
                average_fill = np.nan
        average_fills[i] = average_fill
    return average_fills/mid
 
def get_liq_params(normalized,pair):
    #coin stats
    coinmar = ccxt.coinmarketcap()
    coindata=coinmar.load_markets()
    total_coins=float(coindata[pair]['info']['available_supply'])  #number of coins floating
    order_span = (1,10,20,30,40)
    clip = total_coins/100000                                      #my standard order size 
    ordersizes=np.array([clip* i for i in order_span]+[-clip* i for i in order_span]).astype(int)
    slippage = ((order_fill(normalized,ordersizes,False)-1)*100).round(2)
    #order book
    best_bid = normalized['bid'].max()
    best_ask = normalized['ask'].min()
    mid= (best_bid + best_ask)/2
    spread = best_ask-best_bid
    spread_pct = spread/mid*100
    cross = min(0,spread)
    cross_pct = min(0,spread_pct)
    #arb
    arb_ask = normalized[normalized['ask'] < best_bid]
    arb_bid = normalized[normalized['bid'] > best_ask]
    print('length_bid',len(arb_bid),'len_ask',len(arb_ask))
    if len(arb_bid) == 0:
        arb_dollar = 0
    else:
        end_bid = arb_bid.iloc[-1]['cum_bid_size']
        end_ask = arb_ask.iloc[-1]['cum_ask_size']
        bid_to_empty = end_bid <= end_ask
        if bid_to_empty:
            arb_dollar = -order_fill(normalized,np.array([end_bid]),False)[0]*mid*end_bid + arb_bid.iloc[-1]['cum_bid_size_$']
        else:
            arb_dollar =-order_fill(normalized,np.array([-end_ask]),False)[0]*mid*end_ask + arb_ask.iloc[-1]['cum_ask_size_$']

    result1 = pd.DataFrame([best_bid,best_ask,mid,spread,spread_pct,cross,cross_pct,int(arb_dollar)],
    index=['bid','ask','mid','spread','spread%','cross','cross%','arb_$']).T
    result2 = pd.DataFrame(slippage,index=[str(o) for o in ordersizes]).T
    info = coindata[pair]['info']
    select_info=['symbol','rank','24h_volume_usd','market_cap_usd',
                'available_supply','percent_change_1h','percent_change_24h','percent_change_7d']
    selected_info={key:value for key,value in info.items() if key in select_info}
    result3 = pd.DataFrame(pd.Series(selected_info)).T
    result3.columns=['Coin','Rank','24H % Volume','USD Market Cap M$','Coins Supply M','% 1h','% 24h','% 7d']
    result3['24H % Volume']= round(float(result3['24H % Volume'])/float(result3['USD Market Cap M$'])*100,1)
    result3['USD Market Cap M$'] = round(float(result3['USD Market Cap M$'])/(1000*1000),0)
    result3['Coins Supply M'] = round(float(result3['Coins Supply M'])/(1000*1000),1)
    return [result1,result2,result3]

In [27]:
pair = 'BTC/USD'
exc=get_exchanges_for_pair(pair)

In [28]:
order_books=get_order_books(pair,exc)

In [29]:
order_books

{'bitstamp': {'bids': [[3859.18, 0.97168461],
   [3858.09, 3.0],
   [3857.69, 0.51830525],
   [3857.21, 3.0],
   [3857.08, 0.4],
   [3856.82, 0.5],
   [3856.81, 0.5],
   [3856.51, 10.0],
   [3856.5, 3.8836],
   [3856.4, 3.0],
   [3856.37, 1.1234],
   [3856.08, 2.0],
   [3855.86, 5.2],
   [3855.8, 0.4],
   [3855.31, 4.0],
   [3855.24, 1.0],
   [3854.37, 0.03896],
   [3854.36, 3.217775],
   [3853.9, 3.0],
   [3853.39, 2.0],
   [3853.31, 0.64128204],
   [3853.13, 4.7116],
   [3852.5, 2.59571707],
   [3852.1, 2.729],
   [3852.09, 0.03499997],
   [3851.99, 5.0],
   [3851.97, 5.9368],
   [3851.7, 3.0],
   [3851.07, 0.69833406],
   [3850.87, 0.4],
   [3850.58, 5.3],
   [3849.37, 2.56968551],
   [3849.36, 3.114781],
   [3848.83, 2.82447],
   [3848.06, 4.82152175],
   [3848.05, 0.0078],
   [3848.04, 3.60534342],
   [3847.71, 0.34919163],
   [3846.9, 4.344293],
   [3846.69, 2.376],
   [3846.0, 6.03],
   [3845.6, 0.05],
   [3845.57, 3.80140825],
   [3844.49, 0.00520225],
   [3844.4, 5.4786],
   [

In [30]:
aggregated = aggregate_order_books(order_books)

In [31]:
normalized=normalize_order_book(aggregated)

In [32]:
normalized

,ask,ask_size,cum_ask_size,ask_size_$,cum_ask_size_$,average_ask_fill,asks_exc,bid,bid_size,cum_bid_size,bid_size_$,cum_bid_size_$,average_bid_fill,bids_exc
0.987,3860.809311,50.737284,50.737284,1.959103e+05,1.959103e+05,3861.269311,gemini,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.988,3864.478213,530.534498,581.271783,2.049951e+06,2.245861e+06,3863.702030,liquid,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.989,3868.507525,217.270856,798.542638,8.406235e+05,3.086485e+06,3865.146775,bitstamp,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.990,3872.329702,204.539324,1003.081963,7.920904e+05,3.878575e+06,3866.657994,kraken,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.991,3876.070604,186.715006,1189.796969,7.237513e+05,4.602326e+06,3868.160983,gemini,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.992,3880.194201,137.204985,1327.001954,5.324877e+05,5.134814e+06,3869.484808,liquid,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.993,3884.156675,194.789393,1521.791347,7.565904e+05,5.891404e+06,3871.361406,coinbasepro,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.994,3887.734080,127.019989,1648.811336,4.938200e+05,6.385224e+06,3872.622743,liquid,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.995,3891.427247,152.425980,1801.237316,5.930244e+05,6.978249e+06,3874.141725,coinbasepro,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.996,3895.828253,142.655529,1943.892845,5.557104e+05,7.533959e+06,3875.706990,coinbasepro,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
order_books.keys()

dict_keys(['bitstamp', 'coinbasepro', 'kraken', 'liquid', 'gemini', 'bittrex', 'bitfinex'])

In [34]:
from plotly.offline import plot, iplot , init_notebook_mode
init_notebook_mode(connected=True)

In [35]:
dplot=plot_book(order_books,pair,exc)

In [36]:
dplot2=plot_depth(order_books,pair,exc)

In [37]:
#iplot(dplot)

In [38]:
#iplot(dplot2)

In [44]:
get_liq_params(normalized,pair)[0]

length_bid 26 len_ask 26


,bid,ask,mid,spread,spread%,cross,cross%,arb_$
0,3961.853846,3860.809311,3911.331579,-101.044535,-2.583379,-101.044535,-2.583379,66440.0


In [45]:
get_liq_params(normalized,pair)[1]

length_bid 26 len_ask 26


,175,1757,3515,5272,7030,-175,-1757,-3515,-5272,-7030
0,-1.26,-0.96,-0.37,0.37,1.04,1.17,-0.09,-0.93,-1.65,-2.48


In [46]:
get_liq_params(normalized,pair)[2]

length_bid 26 len_ask 26


,Coin,Rank,24H % Volume,USD Market Cap M$,Coins Supply M,% 1h,% 24h,% 7d
0,BTC,1,13.5,68847.0,17.6,0.05,0.74,1.34


In [56]:
#order book
best_bid = normalized['bid'].max()
best_ask = normalized['ask'].min()
mid= (best_bid + best_ask)/2
spread = best_ask-best_bid
spread_pct = spread/mid*100
cross = min(0,spread)
cross_pct = min(0,spread_pct)
#arb
arb_ask = normalized[normalized['ask'] < best_bid]
arb_bid = normalized[normalized['bid'] > best_ask]
if len(arb_bid) == 0:
    print('no arb')
    arb_dollar = 0
    arb_size = 0
else:
    end_bid = arb_bid.iloc[-1]['cum_bid_size']
    end_ask = arb_ask.iloc[-1]['cum_ask_size']
    bid_to_empty = end_bid <= end_ask
    if bid_to_empty:
        print('selling first')
        arb_size = arb_bid.iloc[-1]['cum_bid_size']
        arb_dollar = -order_fill(normalized,np.array([end_bid]),False)[0]*mid*end_bid + arb_bid.iloc[-1]['cum_bid_size_$']
    else:
        print('buying first')
        arb_size = arb_ask.iloc[-1]['cum_ask_size']
        arb_dollar =-order_fill(normalized,np.array([-end_ask]),False)[0]*mid*end_ask + arb_ask.iloc[-1]['cum_ask_size_$']

result1 = pd.DataFrame([best_bid,best_ask,mid,spread,spread_pct,cross,cross_pct,arb_dollar,arb_size],
index=['bid','ask','mid','spread','spread%','cross','cross%','arb_$','arb_size']).T

selling first


In [57]:
best_bid

3961.8538461538465

In [58]:
best_ask

3860.809311285713

In [65]:
arb_bid

,ask,ask_size,cum_ask_size,ask_size_$,cum_ask_size_$,average_ask_fill,asks_exc,bid,bid_size,cum_bid_size,bid_size_$,cum_bid_size_$,average_bid_fill,bids_exc
1.013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3961.853846,35.188591,35.188591,139426.064035,1.394261e+05,3962.252026,bitfinex
1.012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3957.766667,71.575342,106.763933,283241.033836,4.226671e+05,3958.894041,bitfinex
1.011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3954.305263,54.554716,161.318649,215744.579492,6.384117e+05,3957.457373,bitfinex
1.010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3950.722222,88.743424,250.062073,350633.159104,9.890448e+05,3955.197301,bitfinex
1.009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3946.389474,50.629163,300.691237,199804.618057,1.188849e+06,3953.721659,bitfinex
1.008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3942.617647,41.935313,342.626550,165329.708916,1.354179e+06,3952.347431,bitfinex
1.007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3938.966667,47.206814,389.833364,185950.230181,1.540129e+06,3950.737766,bitfinex
1.006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3934.830000,118.010579,507.843943,464253.331453,2.004383e+06,3946.847753,bitfinex
1.005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3931.217647,73.473134,581.317077,288780.354470,2.293163e+06,3944.771575,bitfinex
1.004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3927.231250,27.915332,609.232408,109629.148324,2.402792e+06,3943.966532,bitfinex


In [66]:
arb_ask

,ask,ask_size,cum_ask_size,ask_size_$,cum_ask_size_$,average_ask_fill,asks_exc,bid,bid_size,cum_bid_size,bid_size_$,cum_bid_size_$,average_bid_fill,bids_exc
0.987,3860.809311,50.737284,50.737284,1.959103e+05,1.959103e+05,3861.269311,gemini,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.988,3864.478213,530.534498,581.271783,2.049951e+06,2.245861e+06,3863.702030,liquid,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.989,3868.507525,217.270856,798.542638,8.406235e+05,3.086485e+06,3865.146775,bitstamp,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.990,3872.329702,204.539324,1003.081963,7.920904e+05,3.878575e+06,3866.657994,kraken,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.991,3876.070604,186.715006,1189.796969,7.237513e+05,4.602326e+06,3868.160983,gemini,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.992,3880.194201,137.204985,1327.001954,5.324877e+05,5.134814e+06,3869.484808,liquid,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.993,3884.156675,194.789393,1521.791347,7.565904e+05,5.891404e+06,3871.361406,coinbasepro,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.994,3887.734080,127.019989,1648.811336,4.938200e+05,6.385224e+06,3872.622743,liquid,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.995,3891.427247,152.425980,1801.237316,5.930244e+05,6.978249e+06,3874.141725,coinbasepro,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.996,3895.828253,142.655529,1943.892845,5.557104e+05,7.533959e+06,3875.706990,coinbasepro,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
arb_size

1539.14622511